# Testing out Linear Programming

In [1]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, value

In [2]:
path = "../data/parts/"

In [3]:
# Load data using pandas
cpus = pd.read_csv(path + "CPU_Data.csv")
gpus = pd.read_csv(path + "GPU_Data.csv")
rams = pd.read_csv(path + "RAM_Data.csv")

motherboards = pd.read_csv(path + "MOBO_Data.csv")
psus = pd.read_csv(path + "PSU_Data.csv")
cases = pd.read_csv(path + "Case_Data.csv")
storage = pd.read_csv(path + "Storage_Data.csv")

games = pd.read_csv("../data/games/top300.csv")

# Return original value if conversion fails
def safe_to_numeric(val):
    try:
        return pd.to_numeric(val)
    except (ValueError, TypeError):
        return val  

# Convert numeric columns from string to numeric types
cpus = cpus.apply(safe_to_numeric)
gpus = gpus.apply(safe_to_numeric)
rams = rams.apply(safe_to_numeric)
cases = cases.apply(safe_to_numeric)
motherboards = motherboards.apply(safe_to_numeric)
psus = psus.apply(safe_to_numeric)
storage = storage.apply(safe_to_numeric)
games = games.apply(safe_to_numeric)

# Normalize CPU & GPU Scores
def normalize_column(df, column_name):
    min_val = df[column_name].min()
    max_val = df[column_name].max()
    df[column_name + "_Normalized"] = (df[column_name] - min_val) / (max_val - min_val)
    return df

cpus = normalize_column(cpus, "Score")
gpus = normalize_column(gpus, "Score")

rams_min = rams["Capacity (GB)"].min()
rams_max = rams["Capacity (GB)"].max()
rams["Capacity (GB)_Normalized"] = 0.01 + 0.99 * (rams["Capacity (GB)"] - rams_min) / (rams_max - rams_min)

In [4]:
# selected_game = 'No Man's Sky'
# selected_game = 'ELDEN RING'
# selected_game = 'Red Dead Redemption 2'
# selected_game = 'Cyberpunk 2077'

# budget = 2000
# budget = 2500
# budget = 3000
budget = 1600

selected_game = 'Counter-Strike 2'
# budget = 1000

# Select the game (copy a game name from top100.csv)
game_data = games[games['name'] == selected_game].iloc[0]

display(game_data)

app_id                                                           730
name                                                Counter-Strike 2
recommendations                                              4248820
genres                                    ['Action', 'Free To Play']
tags               ['FPS', 'Shooter', 'Multiplayer', 'Competitive...
CPU                                                             1300
GPU                                                             4500
memory                                                           8.0
Name: 0, dtype: object

In [5]:
''' ---Filtering--- '''
# Available brands for each part
# CPU: AMD, Intel
# GPU: AMD, ASRock, Asus, EVGA, Gainward, Gigabyte, Intel, MSI, Nvidia, PNY, PowerColor, Sapphire, Zotac
# Ram : Kingston, XPG, Fanxiang, PNY, Corsair, Samsung, Apacer, KLEVV
# Motherboard (Mobo): ASUS, MSI, ASRock, GIGABYTE, Colorful, NZXT
# PSU: 1st Player, Asus, Cooler, Deepcool, Gigabyte, MSI, Silverstone, Thermaltake
# Case: Armaggeddon, 1st Player, Lian Li

# Define allowed brands (customize as needed)
allowed_cpu_brands = ["Intel"]
allowed_gpu_brands = ["Nvidia", "Asus", "Gainward"]
allowed_ram_brands = ["Kingston", "Corsair"]
allowed_psu_brands = ["1st Player", "ASRock"]
allowed_mobo_brands = ["MSI", "ASUS"]
allowed_case_brands = ["1st Player"]

# Filter each DataFrame by brand
cpus_filtered = cpus[cpus["Brand"].isin(allowed_cpu_brands)].reset_index(drop=True)
gpus_filtered = gpus[gpus["Brand"].isin(allowed_gpu_brands)].reset_index(drop=True)
rams_filtered = rams[rams["Brand"].isin(allowed_ram_brands)].reset_index(drop=True)
psus_filtered = psus[psus["Brand"].isin(allowed_psu_brands)].reset_index(drop=True)
motherboards_filtered = motherboards[motherboards["Brand"].isin(allowed_mobo_brands)].reset_index(drop=True)
cases_filtered = cases[cases["Brand"].isin(allowed_case_brands)].reset_index(drop=True)

# Validation to ensure filtered lists are not empty
if cpus.empty or gpus.empty or rams.empty or psus.empty or motherboards.empty or cases.empty:
    raise ValueError("One or more component categories have no items after filtering. Adjust brand filters.")

In [6]:
# stuff to make it easy to pick between filtered and non filtered parts
# and to force the min ram to be 16 gb since that is the standard for having a pc

filter = True
min_ram_16 = True

if filter == True:
    lenCPU = range(len(cpus_filtered))
    lenGPU = range(len(gpus_filtered))
    lenRAM = range(len(rams_filtered))
    lenPSU = range(len(psus_filtered))
    lenMOBO = range(len(motherboards_filtered))
    lenCase = range(len(cases_filtered))

    cpusf = cpus_filtered
    gpusf = gpus_filtered
    ramsf = rams_filtered
    psusf = psus_filtered
    mobosf = motherboards_filtered
    casesf = cases_filtered

else:
    lenCPU = range(len(cpus))
    lenGPU = range(len(gpus))
    lenRAM = range(len(rams))
    lenPSU = range(len(psus))
    lenMOBO = range(len(motherboards))
    lenCase = range(len(cases))

    cpusf = cpus
    gpusf = gpus
    ramsf = rams
    psusf = psus
    mobosf = motherboards
    casesf = cases

### Algoritm B (NEW)

In [7]:
''' Create compatibility dictionary for MOBO'''
# Socket
mobo_cpu_compatibility_dict = {
    (i, j): int(cpu['Socket'] == mobo['Socket'])
    for i, cpu in cpusf.iterrows()
    for j, mobo in mobosf.iterrows()
}

# Size (for Case)
mobo_case_compatibility_dict = {
    (i, j): int(case['Size'] == mobo['Size'])
    for i, case in casesf.iterrows()
    for j, mobo in mobosf.iterrows()
}

# DDR 
mobo_ram_compatibility_dict = {
    (i, j): int(ram['DDR'] == mobo['DDR'])
    for i, ram in ramsf.iterrows()
    for j, mobo in mobosf.iterrows()
}

In [8]:
# --- Creating LP Problem --- 
problem = LpProblem("Desktop_Optimization", LpMaximize)

# --- Define PC Components ---
cpu_vars = [LpVariable(f"cpu_{i}", cat="Binary") for i in lenCPU]
gpu_vars = [LpVariable(f"gpu_{i}", cat="Binary") for i in lenGPU]
psu_vars = [LpVariable(f"psu_{i}", cat="Binary") for i in lenPSU]
mobo_vars = [LpVariable(f"mb_{i}", cat="Binary") for i in lenMOBO]
case_vars = [LpVariable(f"case_{i}", cat="Binary") for i in lenCase]
ram_vars = [LpVariable(f"ram_{i}", cat="Binary") for i in lenRAM]
ram_count = LpVariable("ram_count", lowBound=1, upBound=4, cat="Integer")
ram_count_selected = [LpVariable(f"ram_count_selected_{i}", lowBound=0, upBound=4, cat="Integer") for i in lenRAM] #Auxiliary Variable


''' ---Part Count Constraint--- '''
# Only select exactly one of each major part
problem += lpSum(cpu_vars) == 1, "Select_One_CPU"
problem += lpSum(gpu_vars) == 1, "Select_One_GPU"
problem += lpSum(ram_vars) == 1, "Select_One_RAM"
problem += lpSum(psu_vars) == 1, "Select_One_PSU"
problem += lpSum(mobo_vars) == 1, "Select_One_MOBO"
problem += lpSum(case_vars) == 1, "Select_One_Case"

''' ---Component Constraints--- '''
# GPU x Power Supply
problem += (
    lpSum(gpu_vars[i] * gpusf.iloc[i]["Recommended Power"] for i in lenGPU) <=
    lpSum(psu_vars[i] * psusf.iloc[i]["Wattage"] for i in lenPSU),
    "PSU_Power_Constraint",
)

''' ---Motherboard Constraints--- '''
# Socket 
for i in lenCPU:
    problem += (
        cpu_vars[i] <= lpSum(mobo_cpu_compatibility_dict[(i,j)] * mobo_vars[j] for j in lenMOBO),
        f"CPU_Socket_Compatibility_{i}"
    )

# Case
for i in lenCase:
    problem += (
        case_vars[i] <= lpSum(mobo_case_compatibility_dict[(i,j)] * mobo_vars[j] for j in lenMOBO),
        f"Case_Size_Compatibility_{i}"
    )

# DDR
for i in lenRAM:
    problem += (
        ram_vars[i] <= lpSum(mobo_ram_compatibility_dict[(i,j)] * mobo_vars[j] for j in lenMOBO),
        f"RAM_DDR_Compatibility_{i}"
    )

# RAM Count
problem += ram_count >= 1, "At_Least_One_RAM"
problem += ram_count <= lpSum(mobo_vars[i] * mobosf.iloc[i]["RAM Slot"] for i in lenMOBO)

''' ---RAM Selection Algorithm--- '''
# Link ram_count_selected to ram_vars and ram_count
for i in lenRAM:
    problem += ram_count_selected[i] <= ram_count
    problem += ram_count_selected[i] <= 4 * ram_vars[i]
    problem += ram_count_selected[i] >= ram_count - (1 - ram_vars[i]) * 4
    problem += ram_count_selected[i] >= 0

''' ---Game Constraints--- '''
problem += (
    lpSum(cpu_vars[i] * cpusf.iloc[i]["Score"] for i in lenCPU) >= game_data["CPU"],
    "Game_CPU_Constraint"
)

problem += (
    lpSum(gpu_vars[i] * gpusf.iloc[i]["Score"] for i in lenGPU) >= game_data["GPU"],
    "Game_GPU_Constraint"
)

memory_requirement = int(game_data["memory"])
if (min_ram_16 and memory_requirement <= 16): # If minimum RAM is set to 16GB
    memory_requirement = 16

problem += (
    lpSum(ram_count_selected[i] * ramsf.iloc[i]['Capacity (GB)'] for i in lenRAM) >= memory_requirement,
    "Game_Memory_Constraint"
)

''' ---Cost Function--- '''
total_cost = (
    lpSum(cpu_vars[i] * cpusf.iloc[i]["Price"] for i in lenCPU) +
    lpSum(gpu_vars[i] * gpusf.iloc[i]["Price"] for i in lenGPU) +
    lpSum(ram_count_selected[i] * ramsf.iloc[i]["Price"] for i in lenRAM) +
    lpSum(psu_vars[i] * psusf.iloc[i]["Price"] for i in lenPSU) +
    lpSum(mobo_vars[i] * mobosf.iloc[i]["Price"] for i in lenMOBO) +
    lpSum(case_vars[i] * casesf.iloc[i]["Price"] for i in lenCase)
)

problem += total_cost <= budget, "Budget_Constraint"

''' ---Performance Function---'''
# Parameters
cpu_weight = 0.1
gpu_weight = 0.895
ram_weight = 0.005
dual_channel_bonus_value = 0.02

# Extra variables
dual_channel_bonus_var = LpVariable("dual_channel_bonus_var", cat="Binary")

# Function
total_performance = (
    cpu_weight * lpSum(cpu_vars[i] * cpusf.iloc[i]["Score_Normalized"] for i in lenCPU) +
    gpu_weight * lpSum(gpu_vars[i] * gpusf.iloc[i]["Score_Normalized"] for i in lenGPU) +
    ram_weight * lpSum(ram_count_selected[i] * ramsf.iloc[i]["Capacity (GB)_Normalized"] for i in lenRAM) +
    dual_channel_bonus_value * dual_channel_bonus_var
)

'''
Dual-Channel Activation TLDR:
Solver has to toggle on-off the binary "dual_channel_bonus_var" to figure out
optimal ram selection based on budget & game.
'''
problem += total_performance
problem += ram_count_selected >= 2 * dual_channel_bonus_var, "Dual-Channel Activation Condition"

# --- Solve ---
problem.solve()

1

### Output

In [9]:
# --- Output Results ---

# Get selected components
selected_cpu_idx = [i for i in lenCPU if cpu_vars[i].value() == 1][0]
selected_gpu_idx = [i for i in lenGPU if gpu_vars[i].value() == 1][0]
selected_psu_idx = [i for i in lenPSU if psu_vars[i].value() == 1][0]
selected_mobo_idx = [i for i in lenMOBO if mobo_vars[i].value() == 1][0]
selected_case_idx = [i for i in lenCase if case_vars[i].value() == 1][0]

selected_ram_idx = [i for i in lenRAM if ram_count_selected[i].value() > 0][0]
selected_ram_count = ram_count_selected[selected_ram_idx].value()
selected_ram_model = ramsf.iloc[selected_ram_idx]['Name']
selected_ram_brand = ramsf.iloc[selected_ram_idx]['Brand']

# Part names
selected_cpu = cpusf.iloc[selected_cpu_idx]['Name']
selected_gpu = gpusf.iloc[selected_gpu_idx]['Name']
selected_psu = psusf.iloc[selected_psu_idx]['Name']
selected_mobo = mobosf.iloc[selected_mobo_idx]['Name']
selected_case = casesf.iloc[selected_case_idx]['Name']

# Scores
selected_cpu_score = cpusf.iloc[selected_cpu_idx]["Score"]
selected_gpu_score = gpusf.iloc[selected_gpu_idx]["Score"]

# Prices
cpu_price = cpusf.iloc[selected_cpu_idx]["Price"]
gpu_price = gpusf.iloc[selected_gpu_idx]["Price"]
psu_price = psusf.iloc[selected_psu_idx]["Price"]
mobo_price = mobosf.iloc[selected_mobo_idx]["Price"]
case_price = casesf.iloc[selected_case_idx]["Price"]
ram_unit_price = ramsf.iloc[selected_ram_idx]["Price"]
ram_price = ram_unit_price * selected_ram_count

# Calculate Total RAM capacity
total_ram_capacity = selected_ram_count * ramsf.iloc[selected_ram_idx]["Capacity (GB)"]

# Total cost manually
total_cost_manual = cpu_price + gpu_price + ram_price + psu_price + mobo_price + case_price

# --- Final Display ---
print(f"Requirements for {selected_game}: ")
print(f"CPU Benchmark: {game_data['CPU']}")
print(f"GPU Benchmark: {game_data['GPU']}")
print(f"Memory: {game_data['memory']} GB\n")

print(f"Selected CPU: {selected_cpu}")
print(f"CPU Price: RM {cpu_price}")
print(f"CPU Benchmark Score: {selected_cpu_score}\n")

print(f"Selected GPU: {selected_gpu}")
print(f"GPU Price: RM {gpu_price}")
print(f"GPU Benchmark Score: {selected_gpu_score}\n")

print(f"Selected RAM Model: {selected_ram_brand} - {selected_ram_model}")
print(f"RAM Count: {int(selected_ram_count)}")
print(f"Total RAM Price: RM {ram_price}")
print(f"Total RAM Capacity: {total_ram_capacity} GB\n")

print(f"Selected PSU: {selected_psu}")
print(f"PSU Price: RM {psu_price}\n")

print(f"Selected Motherboard: {selected_mobo}")
print(f"Motherboard Price: RM {mobo_price}\n")

print(f"Selected Case: {selected_case}")
print(f"Case Price: RM {case_price}\n")

print(f"Total Cost: RM {total_cost_manual:.2f}")

print("\n❗ Keep in mind that hardware prices fluctuate often and that this list may not be accurate ❗")

Requirements for Counter-Strike 2: 
CPU Benchmark: 1300
GPU Benchmark: 4500
Memory: 8.0 GB

Selected CPU: Core i3-12100F
CPU Price: RM 339
CPU Benchmark Score: 14090

Selected GPU: Asus GTX 1660 6GB Phoenix
GPU Price: RM 400
GPU Benchmark Score: 11667

Selected RAM Model: Kingston - HyperX FURY
RAM Count: 2
Total RAM Price: RM 244.0
Total RAM Capacity: 32.0 GB

Selected PSU: ACK Standard
PSU Price: RM 155

Selected Motherboard: Prime H610M-K
Motherboard Price: RM 299

Selected Case: Black Sir B5-M
Case Price: RM 135

Total Cost: RM 1572.00

❗ Keep in mind that hardware prices fluctuate often and that this list may not be accurate ❗
